In [1]:
import mlflow
import os
import pandas as pd
import pickle
import numpy as np

In [2]:
!mlflow --version 

mlflow, version 2.20.3


In [3]:
Homework_Path = os.getcwd()
Data_Path = "."

In [4]:
os.chdir(Data_Path)
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet
os.listdir()
os.chdir(Homework_Path)

--2025-03-20 06:57:08--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.119, 18.239.238.212, 18.239.238.133, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427002 (1.4M) [binary/octet-stream]
Saving to: ‘green_tripdata_2023-01.parquet.3’

green_tripdata_2023 100%[===================>]   1.36M  1.92MB/s    in 0.7s    

2025-03-20 06:57:09 (1.92 MB/s) - ‘green_tripdata_2023-01.parquet.3’ saved [1427002/1427002]

--2025-03-20 06:57:10--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.119, 18.239.238.212, 18.239.238.133, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.119|:443... connected.
HTTP

In [5]:
ls -ltrh green*.parquet | wc -l

4


Train a model with autolog - Q3

In [6]:
import mlflow
import mlflow.sklearn

mlflow.sklearn.autolog()

In [7]:
train_path = "./green_tripdata_2023-01.parquet"
val_path = "./green_tripdata_2023-02.parquet"
test_path = "./green_tripdata_2023-03.parquet"

df_train = pd.read_parquet(train_path)
df_val = pd.read_parquet(val_path)
df_test = pd.read_parquet(test_path)

df_train.head()
df_val.head()
df_test.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-03-01 00:25:10,2023-03-01 00:35:47,N,1.0,82,196,1.0,2.36,13.5,1.0,0.5,0.00,0.0,NaN,1.0,16.00,2.0,1.0,0.00
1,2,2023-03-01 00:14:29,2023-03-01 00:25:04,N,1.0,7,7,1.0,0.78,-6.5,-1.0,-0.5,0.00,0.0,NaN,-1.0,-9.00,3.0,1.0,0.00
2,2,2023-03-01 00:14:29,2023-03-01 00:25:04,N,1.0,7,7,1.0,0.78,6.5,1.0,0.5,0.00,0.0,NaN,1.0,9.00,3.0,1.0,0.00
3,2,2023-02-28 22:59:46,2023-02-28 23:08:38,N,1.0,166,74,1.0,1.66,11.4,1.0,0.5,2.78,0.0,NaN,1.0,16.68,1.0,1.0,0.00
4,2,2023-03-01 00:54:03,2023-03-01 01:03:14,N,1.0,236,229,1.0,3.14,15.6,1.0,0.5,4.17,0.0,NaN,1.0,25.02,1.0,1.0,2.75


In [8]:
import datetime
from sklearn.feature_extraction import DictVectorizer

def create_dataframe(filepath):
    df = pd.read_parquet(train_path)
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df['pickup_day'] = df.lpep_pickup_datetime.apply(lambda x: x.weekday())
    df['pickup_hour'] = df.lpep_pickup_datetime.apply(lambda x: x.hour)
    df['pickup_min'] = df.lpep_pickup_datetime.apply(lambda x: x.minute)

    
    chosen_cols = ['duration','PULocationID','DOLocationID','trip_distance','pickup_day','pickup_hour','pickup_min']
    df = df[chosen_cols]

    return df

def preprocess_dataframe(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']

    numerical = ['trip_distance','pickup_day','pickup_hour','pickup_min']
    
    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, dv

def dump_pickle(obj, filename: str):
    with open(filename, "wb") as f_out:
        return pickle.dump(obj, f_out)

In [9]:
dv = DictVectorizer()

df_train = create_dataframe(train_path)
df_val = create_dataframe(val_path)
df_test = create_dataframe(test_path)

X_train, dv = preprocess_dataframe(df_train,dv, fit_dv=True)
X_val, _ = preprocess_dataframe(df_val,dv, fit_dv=False)
X_test, _ = preprocess_dataframe(df_test,dv, fit_dv=False)

print(f"X_train contains: {np.shape(X_train)[0]} entries, and {np.shape(X_train)[1]} columns")
print(f"X_val contains: {np.shape(X_val)[0]} entries, and {np.shape(X_val)[1]} columns")
print(f"X_test contains: {np.shape(X_test)[0]} entries, and {np.shape(X_test)[1]} columns")

X_train contains: 65946 entries, and 5705 columns
X_val contains: 65946 entries, and 5705 columns
X_test contains: 65946 entries, and 5705 columns


In [10]:
y_train = df_train['duration']
y_val = df_val['duration']
y_test = df_test['duration']


dump_pickle(dv, os.path.join(Data_Path, "dv.pkl"))
dump_pickle((X_train, y_train), os.path.join(Data_Path, "train.pkl"))
dump_pickle((X_val, y_val), os.path.join(Data_Path, "val.pkl"))
dump_pickle((X_test, y_test), os.path.join(Data_Path, "test.pkl"))


In [12]:
import os
import pickle
import mlflow
import mlflow.sklearn


mlflow.sklearn.autolog()



from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

data_path = "."

X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

mlflow.sklearn.autolog()
with mlflow.start_run():
    rf = RandomForestRegressor(max_depth=10, random_state=0)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    print("RMSE:", rmse)


2025/03/20 07:07:25 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
2025/03/20 07:07:53 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI, but it was currently set to file:///workspaces/MLOps/mlruns. Perhaps you forgot to set the tracking URI to the running MLflow server. To set the tracking URI, use either of the following methods:
1. Set the MLFLOW_TRACKING_URI environment variable to the desired tracking URI. `export MLFLOW_TRACKING_URI=http://localhost:5000`
2. Set the tracking URI programmatically by calling `mlflow.set_tracking_uri`. `mlflow.set_tracking_uri('http://localhost:5000')`


RMSE: 4.877393542248552


Launch the tracking server locally -Q4

In [13]:
database_uri = "sqlite:///mlflow.db"
experiment_name = "nyc-taxi-ride-prediction-single"

mlflow.set_tracking_uri(database_uri) 
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='/workspaces/MLOps/mlruns/1', creation_time=1741584546996, experiment_id='1', last_update_time=1741584546996, lifecycle_stage='active', name='nyc-taxi-ride-prediction-single', tags={}>

In [17]:
import sys

# Override sys.argv to only include the script name (or desired options)
sys.argv = ['hpo.py']  # or ['hpo.py', '--data_path', './output', '--num_trials', '15']

# Now call the Click command with standalone_mode=False to avoid automatic error handling
run_optimization(standalone_mode=False)


FileNotFoundError: [Errno 2] No such file or directory: './output/train.pkl'

In [22]:
import os
import pickle
import click
import mlflow
import numpy as np
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll import scope
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("random-forest-hyperopt")

def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run_optimization(data_path=".", num_trials=15):

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    def objective(params):
        # Start a nested MLflow run to log each hyperopt trial
        with mlflow.start_run(nested=True):
            mlflow.log_params(params)
            
            rf = RandomForestRegressor(**params)
            rf.fit(X_train, y_train)
            y_pred = rf.predict(X_val)
            rmse = mean_squared_error(y_val, y_pred, squared=False)
            
            mlflow.log_metric("rmse", rmse)
            
        return {'loss': rmse, 'status': STATUS_OK}

    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        'random_state': 42
    }

    rstate = np.random.default_rng(42)  # for reproducibility
    fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_trials,
        trials=Trials(),
        rstate=rstate
    )

if __name__ == '__main__':
    run_optimization()


  0%|                                                                                                         | 0/15 [00:00<?, ?trial/s, best loss=?]

2025/03/20 07:28:15 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

2025/03/20 07:28:41 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpm9g0sv80/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 



🏃 View run angry-ox-944 at: http://127.0.0.1:5000/#/experiments/655456287014535769/runs/8938ac725cce4f558793b937d977a7f9                            

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/655456287014535769                                                                        

  7%|█████▎                                                                          | 1/15 [00:29<06:56, 29.78s/trial, best loss: 4.669037866079264]

2025/03/20 07:28:44 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

2025/03/20 07:28:48 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpk4g6vira/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 



🏃 View run redolent-sow-569 at: http://127.0.0.1:5000/#/experiments/655456287014535769/runs/639d9048c1354f6d97d5bbb6f1a2dee3                        

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/655456287014535769                                                                        

 13%|██████████▋                                                                     | 2/15 [00:35<03:23, 15.65s/trial, best loss: 4.669037866079264]

...

🏃 View run wise-bat-10 at: http://127.0.0.1:5000/#/experiments/655456287014535769/runs/172410025dec4874acdc22f7cab9a9ae                             

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/655456287014535769                                                                        

 87%|████████████████████████████████████████████████████████████████████▍          | 13/15 [03:22<00:23, 11.79s/trial, best loss: 4.108006448090099]

2025/03/20 07:31:37 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

2025/03/20 07:31:49 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpito1n8me/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 



🏃 View run rambunctious-ram-599 at: http://127.0.0.1:5000/#/experiments/655456287014535769/runs/80369df144b04cf29755d6649361ade8                    

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/655456287014535769                                                                        

 93%|█████████████████████████████████████████████████████████████████████████▋     | 14/15 [03:35<00:12, 12.29s/trial, best loss: 4.108006448090099]

2025/03/20 07:31:51 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'

2025/03/20 07:32:09 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpyr66a4ov/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 



🏃 View run overjoyed-steed-139 at: http://127.0.0.1:5000/#/experiments/655456287014535769/runs/81d59ed7e5ce4445a1a2231af000a17e                     

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/655456287014535769                                                                        

100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [03:55<00:00, 15.70s/trial, best loss: 4.108006448090099]


Q6 Promote the best model to the model registry


In [1]:
import os
import pickle
import mlflow
from mlflow.tracking import MlflowClient
import numpy as np
from sklearn.metrics import mean_squared_error
import mlflow.sklearn


In [13]:
def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


test_data_path = "."  
X_test, y_test = load_pickle(os.path.join(test_data_path, "test.pkl"))


In [15]:
import mlflow

experiment_name = "random-forest-best-model"
mlflow.set_experiment(experiment_name)


2025/03/20 10:04:30 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-model' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/576275541145509561', creation_time=1742465070564, experiment_id='576275541145509561', last_update_time=1742465070564, lifecycle_stage='active', name='random-forest-best-model', tags={}>

In [5]:
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    # Create the experiment if it doesn't exist.
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

print("Using experiment id:", experiment_id)


Traceback (most recent call last):
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 328, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 422, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1368, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1361, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/mlflow/utils/file_utils.py", line 310, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.exceptions.Missi

Using experiment id: 655456287014535769


In [7]:
experiment_name = "random-forest-hyperopt"
mlflow.set_experiment(experiment_name)
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    # Create experiment if it doesn't exist.
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = experiment.experiment_id

print("Using experiment id:", experiment_id)


Traceback (most recent call last):
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 328, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 422, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1368, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py", line 1361, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/codespace/anaconda3/lib/python3.9/site-packages/mlflow/utils/file_utils.py", line 310, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.exceptions.Missi

Using experiment id: 655456287014535769


In [8]:
runs = client.search_runs(
    experiment_ids=[experiment_id],
    filter_string="metrics.rmse >= 0",
    order_by=["metrics.rmse ASC"],
    max_results=5
)


In [11]:
best_rmse = float("inf")
best_run_id = None

for run in runs:
    run_id = run.info.run_id
    model_uri = f"http://127.0.0.1:5000/#/experiments/655456287014535769/model"
    
    # Load the model from MLflow
    model = mlflow.sklearn.load_model(model_uri)
    
    # Predict on the test set and compute RMSE
    y_pred = model.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"Run ID: {run_id}, Test RMSE: {rmse_test}")
    
    if rmse_test < best_rmse:
        best_rmse = rmse_test
        best_run_id = run_id

print("Best run ID:", best_run_id)
print("Best test RMSE:", best_rmse)


ValueError: not enough values to unpack (expected 2, got 1)

In [21]:
import os
import pickle
import click
import mlflow

from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Experiment names
HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
BEST_MODEL_EXPERIMENT_NAME = "random-forest-best-models"
# Only include these RF hyperparameters from the run's parameters
RF_PARAMS = ['max_depth', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 'random_state']

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(BEST_MODEL_EXPERIMENT_NAME)
mlflow.sklearn.autolog()  # autologging enabled for this experiment

def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

def train_and_log_model(data_path, params):
    # Load training, validation, and test datasets
    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))
    X_test, y_test = load_pickle(os.path.join(data_path, "test.pkl"))

    # Filter parameters to only those in RF_PARAMS and cast them to int
    filtered_params = {}
    for param in RF_PARAMS:
        if param in params:
            try:
                filtered_params[param] = int(params[param])
            except ValueError:
                # if conversion to int fails, try float or use as is
                try:
                    filtered_params[param] = float(params[param])
                except ValueError:
                    filtered_params[param] = params[param]
    # Start an MLflow run for this training
    with mlflow.start_run():
        rf = RandomForestRegressor(**filtered_params)
        rf.fit(X_train, y_train)

        # Evaluate on the validation set and log the metric
        val_rmse = mean_squared_error(y_val, rf.predict(X_val), squared=False)
        mlflow.log_metric("val_rmse", val_rmse)
        # Evaluate on the test set and log the metric
        test_rmse = mean_squared_error(y_test, rf.predict(X_test), squared=False)
        mlflow.log_metric("test_rmse", test_rmse)
        # The trained model is also logged (via autolog)


def run_register_model(data_path=".", top_n=5):
    client = MlflowClient()

    # Retrieve the hyperopt experiment by name
    hpo_experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
    if hpo_experiment is None:
        raise Exception(f"Experiment '{HPO_EXPERIMENT_NAME}' not found.")
    
    # Get the top_n runs sorted by rmse (lowest first) from the hyperopt experiment
    hpo_runs = client.search_runs(
        experiment_ids=[hpo_experiment.experiment_id],
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=top_n,
        order_by=["metrics.rmse ASC"]
    )
    if not hpo_runs:
        raise Exception("No runs found in the hyperopt experiment.")

    # For each of the top_n hyperopt runs, retrain the model with those parameters
    # and log results into the best-models experiment.
    for run in hpo_runs:
        # Use only the desired parameters from the run
        run_params = {k: v for k, v in run.data.params.items() if k in RF_PARAMS}
        train_and_log_model(data_path=data_path, params=run_params)

    # Now search in the BEST_MODEL_EXPERIMENT_NAME for the run with the lowest test RMSE
    best_experiment = client.get_experiment_by_name(BEST_MODEL_EXPERIMENT_NAME)
    best_runs = client.search_runs(
        experiment_ids=[best_experiment.experiment_id],
        run_view_type=ViewType.ACTIVE_ONLY,
        order_by=["metrics.test_rmse ASC"],
        max_results=1
    )
    if not best_runs:
        raise Exception("No runs found in the best-models experiment.")

    best_run = best_runs[0]
    best_run_id = best_run.info.run_id
    best_test_rmse = best_run.data.metrics["test_rmse"]

    print("Best run ID:", best_run_id)
    print("Best test RMSE:", best_test_rmse)

    # Register the best model to the model registry.
    model_uri = f"runs:/{best_run_id}/model"
    model_name = "RandomForest_Best_Model"
    registered_model = mlflow.register_model(model_uri, model_name)

    print("Model registered as:", registered_model.name)
    print("Registered model version:", registered_model.version)

if __name__ == '__main__':
    run_register_model()


2025/03/20 10:12:50 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
2025/03/20 10:13:05 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpu7hsvd4t/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 


🏃 View run painted-bear-697 at: http://127.0.0.1:5000/#/experiments/472326644643157653/runs/3d2f9ae98ab1418e8cb1590b0fcd0865
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/472326644643157653


2025/03/20 10:13:10 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
2025/03/20 10:13:29 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpzh8kt6u4/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 


🏃 View run bedecked-lynx-842 at: http://127.0.0.1:5000/#/experiments/472326644643157653/runs/271c15a1a8de4371a7ac993a9b7cc808
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/472326644643157653


2025/03/20 10:13:32 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
2025/03/20 10:14:02 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp_ej_buoj/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 


...

2025/03/20 10:14:05 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
2025/03/20 10:14:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpjeyp8imz/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 


🏃 View run bouncy-shoat-359 at: http://127.0.0.1:5000/#/experiments/472326644643157653/runs/b25e5fc7310b4cc1b55d3292953e7d2f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/472326644643157653


2025/03/20 10:14:20 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'toarray'
2025/03/20 10:14:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpe8t92vii/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
Successfully registered model 'RandomForest_Best_Model'.
2025/03/20 10:14:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForest_Best_Model, version 1


🏃 View run gregarious-cod-528 at: http://127.0.0.1:5000/#/experiments/472326644643157653/runs/170ec091d9c4461b9870688040deee41
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/472326644643157653
Best run ID: 3d2f9ae98ab1418e8cb1590b0fcd0865
Best test RMSE: 4.108006448090099
Model registered as: RandomForest_Best_Model
Registered model version: 1


Created version '1' of model 'RandomForest_Best_Model'.
